# Makes some test on the PhotonCountingLayer

The second derivatives of the chi matrix are used to evaluate the expected number of photons in each channels

Here we test the photon counter layer on different states

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>

nonlinearxwaves@gmail.com<br>
@created 15 december 2020<br>
@version 23 sep 2023

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warning messages

In [2]:
import numpy as np
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
import matplotlib.pyplot as plt

In [3]:
tf.keras.backend.clear_session()

In [4]:
np.set_printoptions(precision=4)

In [5]:
tf_real='float64'

In [6]:
tf.keras.backend.set_floatx(tf_real)

## Dimension (number of modes times 2, N=2n)

In [7]:
N = 4

## Build vacuum by the Gaussian state

In [8]:
vacuum = ps.VacuumLayer(N, dtype=tf_real)

In [9]:
vacuum.dtype

'float64'

## Squeezed mode on mode 0

In [10]:
ra=0.1;
phia=np.pi/2;

In [11]:
squeezer=ps.SingleModeSqueezerLayer(N, r_np=ra, theta_np=phia, n_squeezed=0, trainable=False, dtype=tf_real)

In [12]:
squeezer.dtype

'float64'

## Coherent state (Displacer on mode 1)

In [13]:
A_np = 1000
lambda_np = np.pi/2
dinput=np.zeros((N,1));
dinput[2]=np.sqrt(2)*A_np*np.cos(lambda_np);
dinput[3]=np.sqrt(2)*A_np*np.sin(lambda_np);
displacer=ps.DisplacementLayerConstant(dinput, dtype=tf_real)

In [14]:
displacer.dtype

'float64'

### Dummy training points (not used for training in this example)

In [15]:
Nbatch=10
gtarget=np.eye(N)
dtarget=2.4*np.ones((N,1))
xtrain = np.random.rand(Nbatch, N)-0.5
ytrain =np.zeros_like(xtrain)
dtrain = np.zeros((Nbatch,N))
gtrain = np.zeros((Nbatch,N,N))
for j in range(Nbatch):
    for i in range(N):
        dtrain[j,i]=dtarget[i]
        for k in range(N):
            gtrain[j,i,k]=gtarget[i,k]

In [16]:
dtrain.dtype

dtype('float64')

# Photon counter on the vacuum state

In [17]:
xin = tf.keras.layers.Input(N, dtype=tf_real)
chir, chii = vacuum(xin)
model = tf.keras.Model(inputs = xin, outputs=[chir, chii])

In [18]:
model.dtype

'float64'

In [19]:
xin.dtype

tf.float64

In [20]:
vacuum.dtype

'float64'

## Evaluate the number of particles without using the photon counter layer

### Evaluate the derivatives by the jacobian and extract the diagonal part 

In [21]:
x = tf.Variable(np.zeros((1,N)),dtype=tf_real)
with tf.GradientTape() as t1:
    with tf.GradientTape() as t2:
        cr, _ = model(x) # calll the model
    cr_x = t2.gradient(cr, x) # call the gradient
cr_xx=t1.jacobian(cr_x,x) # call the jacobian
print(cr) # the model is a scalar
print(cr_x) # the gradient is a vector 
print(cr_xx) # the jacobian is a tensor with four indices the first two is the deriver, the second two is the variable
# we need to extract the diagonal of the jacobian 
tmp1 = tf.reshape(cr_xx, [N,N]) # first reshape as a matrix
print(tmp1)
lapls=tf.linalg.diag_part(tmp1); # then take the diagonal part
print(lapls)

tf.Tensor([[1.]], shape=(1, 1), dtype=float64)
tf.Tensor([[0. 0. 0. 0.]], shape=(1, 4), dtype=float64)
tf.Tensor(
[[[[-0.5  0.   0.   0. ]]

  [[ 0.  -0.5  0.   0. ]]

  [[ 0.   0.  -0.5  0. ]]

  [[ 0.   0.   0.  -0.5]]]], shape=(1, 4, 1, 4), dtype=float64)
tf.Tensor(
[[-0.5  0.   0.   0. ]
 [ 0.  -0.5  0.   0. ]
 [ 0.   0.  -0.5  0. ]
 [ 0.   0.   0.  -0.5]], shape=(4, 4), dtype=float64)
tf.Tensor([-0.5 -0.5 -0.5 -0.5], shape=(4,), dtype=float64)


### Combine the secon derivatives to hava d_qq+d_pp and return the number of photons

In [22]:
RQ, RP, _ = ps.RQRP(N);
-0.5*(np.matmul(lapls, tf.constant(RQ))+np.matmul(lapls, tf.constant(RP)))-0.5

array([0., 0.])

Remark : The vacuum state has on average 0 photons (zero point energy is not present in normally ordered operator)

## Use the photon counting layer

In [23]:
photon_counter=ps.PhotonCountingLayer(N,dtype=tf_real) # define the layer
n_out = photon_counter(chir,chir, model);  # define the output tensor
Nphoton = tf.keras.Model(inputs = xin, outputs=n_out) # define the model with inputs and ouputs
print(Nphoton(xtrain)); 

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float64)


In [24]:
photon_counter.dtype

'float64'

In [25]:
model.dtype

'float64'

# Photon counting on the coherent state

In [26]:
xin = tf.keras.layers.Input(N, dtype=tf_real)
x0, a0 = displacer(xin)
chir, chii = vacuum(x0,a0)
model = tf.keras.Model(inputs = xin, outputs=[chir, chii])
photon_counter=ps.PhotonCountingLayer(N, dtype=tf_real) # define the layer
n_out = photon_counter(chir,chir, model);  # define the output tensor
Nphoton = tf.keras.Model(inputs = xin, outputs=n_out) # define the model with inputs and ouputs
print(Nphoton(xtrain)); 

tf.Tensor([[      0. 1000000.]], shape=(1, 2), dtype=float64)


The ouput is 0 for mode 0 (vacuum) and A_np^2 for mode 1 (coherent state)

# Photon counting on the coherent state and squeezed vacuum

In [27]:
xin = tf.keras.layers.Input(N, dtype=tf_real)
x0, a0 = squeezer(xin)
x1, a1 = displacer(x0,a0)
chir, chii = vacuum(x1,a1)
model = tf.keras.Model(inputs = xin, outputs=[chir, chii])
photon_counter=ps.PhotonCountingLayer(N, dtype=tf_real) # define the layer
n_out = photon_counter(chir,chir, model);  # define the output tensor
Nphoton = tf.keras.Model(inputs = xin, outputs=n_out) # define the model with inputs and ouputs
print(Nphoton(xtrain)); 

tf.Tensor([[1.0033e-02 1.0000e+06]], shape=(1, 2), dtype=float64)


The number of photons does not change in the squeezed vacuum 